<a href="https://colab.research.google.com/github/metateca/app/blob/main/io.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.cluster as cluster
import networkx as nx
from IPython.display import display, Markdown
import os
from google.colab import drive

# --- Configurações e Constantes ---
DEFAULT_PALETTE = "viridis"
FIGSIZE = (12, 8)

sns.set(style="darkgrid")
plt.rcParams.update({
    'figure.facecolor': 'black',
    'axes.facecolor': 'black',
    'axes.edgecolor': 'white',
    'axes.labelcolor': 'cyan',
    'xtick.color': 'white',
    'ytick.color': 'white',
    'text.color': 'white',
    'grid.color': 'gray',
    'grid.linestyle': '--',
    'legend.facecolor': 'black',
    'legend.edgecolor': 'white',
    'figure.titlesize': 20,
    'axes.titlesize': 16,
    'axes.labelsize': 14,
    'xtick.labelsize': 12,
    'ytick.labelsize': 12,
    'legend.fontsize': 12,
})

# --- Funções Auxiliares ---

def mount_google_drive():
    """Monta o Google Drive."""
    drive.mount('/content/drive')

def get_drive_path(relative_path):
    """Retorna o caminho completo no Drive."""
    return os.path.join('/content/drive/MyDrive', relative_path)

def ensure_directory_exists_on_drive(relative_path):
    """Cria diretório no Drive, se não existir."""
    drive_path = get_drive_path(relative_path)
    if not os.path.exists(drive_path):
        os.makedirs(drive_path, exist_ok=True)
        print(f"Diretório criado: {drive_path}")
    return drive_path

def create_figure():
    """Cria uma figura com tamanho padrão."""
    return plt.figure(figsize=FIGSIZE)

def save_fig(fig, filename, drive_folder_path):
    """Salva a figura no Drive."""
    try:
        filepath = os.path.join(drive_folder_path, filename)
        fig.savefig(filepath, dpi=300, bbox_inches='tight')
        print(f"Gráfico salvo: {filepath}")
    except Exception as e:
        print(f"Erro ao salvar '{filename}': {e}")
    finally:
        plt.close(fig)

# --- Funções de Visualização ---

def plot_boxplot(df, x_col, y_col, title, filename, drive_folder_path, hue=None):
    """Plota boxplots."""
    try:
        if x_col not in df.columns or y_col not in df.columns:
            raise ValueError(f"Colunas '{x_col}' ou '{y_col}' não encontradas.")
        fig = create_figure()
        sns.boxplot(x=x_col, y=y_col, data=df, palette=DEFAULT_PALETTE, hue=hue)
        plt.title(title)
        plt.xlabel(x_col.replace('_', ' '))
        plt.ylabel(y_col.replace('_', ' '))
        if hue and hue in df.columns:
            plt.legend(title=hue.replace('_', ' '), loc='upper right')
        save_fig(fig, filename, drive_folder_path)
    except (ValueError, KeyError, Exception) as e:
        print(f"Erro em plot_boxplot: {e}")

def plot_violin(df, x_col, y_col, title, filename, drive_folder_path, hue=None, split=False):
    """Plota gráfico de violino."""
    try:
        if x_col not in df.columns or y_col not in df.columns:
            raise ValueError(f"Colunas '{x_col}' ou '{y_col}' não encontradas.")
        fig = create_figure()
        sns.violinplot(x=x_col, y=y_col, data=df, palette=DEFAULT_PALETTE, hue=hue, split=split)
        plt.title(title)
        plt.xlabel(x_col.replace('_', ' '))
        plt.ylabel(y_col.replace('_', ' '))
        if hue and hue in df.columns:
            plt.legend(title=hue.replace('_', ' '), loc='upper right')
        save_fig(fig, filename, drive_folder_path)
    except (ValueError, KeyError, Exception) as e:
        print(f"Erro em plot_violin: {e}")

def plot_kde(df, x_col, title, filename, drive_folder_path, hue=None, multiple="layer"):
    """Plota gráfico de densidade KDE."""
    try:
        if x_col not in df.columns:
            raise ValueError(f"Coluna '{x_col}' não encontrada.")
        fig = create_figure()
        sns.kdeplot(data=df, x=x_col, hue=hue, palette=DEFAULT_PALETTE, fill=True, multiple=multiple)
        plt.title(title)
        plt.xlabel(x_col.replace('_', ' '))
        if hue and hue in df.columns:
            plt.legend(title=hue.replace('_', ' '), loc='upper right')
        save_fig(fig, filename, drive_folder_path)
    except (ValueError, KeyError, Exception) as e:
        print(f"Erro em plot_kde: {e}")

def plot_line(df, x_col, y_col, title, filename, drive_folder_path, hue=None):
    """Plota gráfico de linha."""
    try:
        if x_col not in df.columns or y_col not in df.columns:
            raise ValueError(f"Colunas '{x_col}' ou '{y_col}' não encontradas.")
        fig = create_figure()
        sns.lineplot(x=x_col, y=y_col, data=df, hue=hue, palette=DEFAULT_PALETTE)
        plt.title(title)
        plt.xlabel(x_col.replace('_', ' '))
        plt.ylabel(y_col.replace('_', ' '))
        if hue and hue in df.columns:
            plt.legend(title=hue.replace('_', ' '), loc='upper right')
        save_fig(fig, filename, drive_folder_path)
    except (ValueError, KeyError, Exception) as e:
        print(f"Erro em plot_line: {e}")

# --- Funções de Análise --- (Mantidas, mas não diretamente usadas para plotagem)

def analyze_clusters(df, features, n_clusters=4):
    """Análise de agrupamento (k-means)."""
    print("\n--- Análise de Agrupamento (K-Means) ---")
    try:
        if not all(feature in df.columns for feature in features):
            raise ValueError("Nem todas as features estão no DataFrame.")
        kmeans = cluster.KMeans(n_clusters=n_clusters, random_state=42, n_init='auto')
        df['cluster'] = kmeans.fit_predict(df[features])
        print("Distribuição dos Clusters:\n", df['cluster'].value_counts())
        print("Características dos Clusters (Médias):\n", df.groupby('cluster')[features].mean())
        return df
    except (ValueError, Exception) as e:
        print(f"Erro em analyze_clusters: {e}")
        return df

def analyze_social_network(df, interaction_threshold=3):
    """Análise de rede social."""
    print("\n--- Análise de Rede Social ---")
    try:
        if 'aluno_id' not in df.columns or 'interacoes_colegas' not in df.columns:
            raise ValueError("'aluno_id' e 'interacoes_colegas' são necessários.")
        graph = nx.Graph()
        graph.add_nodes_from(df['aluno_id'])
        for i, row_i in df.iterrows():
            for j, row_j in df.iterrows():
                if i < j:
                    if abs(row_i['interacoes_colegas'] - row_j['interacoes_colegas']) <= interaction_threshold:
                        graph.add_edge(row_i['aluno_id'], row_j['aluno_id'])
        degree_centrality = nx.degree_centrality(graph)
        df['centralidade_grau'] = df['aluno_id'].map(degree_centrality)
        df['centralidade_grau'].fillna(0, inplace=True)
        return graph, df  # Retorna o grafo e o DataFrame
    except (ValueError, Exception) as e:
        print(f"Erro em analyze_social_network: {e}")
        return nx.Graph(), df

def create_recommendations(df):
    """Cria recomendações (simples)."""
    print("\n--- Sistema de Recomendação (Simulado) ---")
    disciplinas = ['disciplina_1', 'disciplina_2', 'disciplina_3']
    df['recomendacao'] = ''
    for index, row in df.iterrows():
        available = [d for d in disciplinas if row[d] == 0]
        df.loc[index, 'recomendacao'] = np.random.choice(available or disciplinas)
    print("Recomendações (5 primeiros):\n", df[['aluno_id', 'recomendacao']].head())
    return df

# --- Função para Gerar Dados Sintéticos (Opcional) ---
def generate_synthetic_data(n_alunos=40):
    """Gera dados sintéticos se o CSV falhar."""
    np.random.seed(42)
    data = {
        'aluno_id': range(1, n_alunos + 1),
        'grupo': np.random.choice(['Com_Agentes', 'Sem_Agentes'], size=n_alunos, p=[0.5, 0.5]),
        'disciplina_1': np.random.choice([0, 1], size=n_alunos, p=[0.7, 0.3]),
        'disciplina_2': np.random.choice([0, 1], size=n_alunos, p=[0.6, 0.4]),
        'disciplina_3': np.random.choice([0, 1], size=n_alunos, p=[0.8, 0.2]),
        'participacao_forum': np.random.exponential(scale=2, size=n_alunos),
        'interacoes_colegas': np.random.poisson(lam=5, size=n_alunos),
        'desempenho': np.random.normal(loc=7, scale=1.5, size=n_alunos),
        'satisfacao': np.random.randint(1, 6, size=n_alunos),
        'pertencimento': np.random.randint(1, 6, size=n_alunos),
        'centralidade_grau': np.random.rand(n_alunos)
    }
    df = pd.DataFrame(data)
    df.loc[df['grupo'] == 'Com_Agentes', 'participacao_forum'] *= 1.5
    df.loc[df['grupo'] == 'Com_Agentes', 'interacoes_colegas'] += 2
    df.loc[df['grupo'] == 'Com_Agentes', 'desempenho'] += 1
    df.loc[df['grupo'] == 'Com_Agentes', 'satisfacao'] = np.clip(df.loc[df['grupo'] == 'Com_Agentes', 'satisfacao'] + 1, 1, 5)
    df.loc[df['grupo'] == 'Com_Agentes', 'pertencimento'] = np.clip(df.loc[df['grupo'] == 'Com_Agentes', 'pertencimento'] + 1, 1, 5)
    df.loc[df['grupo'] == 'Com_Agentes', 'centralidade_grau'] *= 1.5
    return df

# --- Função Principal (main) ---
if __name__ == "__main__":
    mount_google_drive()
    graficos_drive_path = ensure_directory_exists_on_drive('graficos')

    display(Markdown("# Análise do Modelo Metateca"))

    # --- Carregar Dados (ou Gerar Sintéticos) ---
    data_url = "https://docs.google.com/spreadsheets/d/1UwNS3dBbamoyVoygXH0RYfucYwoAHnYL-Q5F8Q2ZyRo/export?format=csv"
    try:
        df = pd.read_csv(data_url)
        df.columns = [col.strip().lower().replace(" ", "_") for col in df.columns]
        df = df.rename(columns={'estudante_id': 'aluno_id'})
        for col in ['habilidade_previa', 'tempo_tarefa', 'qualidade_escrita',
                    'resolucao_problemas', 'pensamento_critico', 'num_prompts', 'satisfacao']:
            if col in df.columns:
                df[col] = pd.to_numeric(df[col], errors='coerce')
        df.dropna(inplace=True)
    except Exception as e:
        print(f"Erro ao carregar dados: {e}. Usando dados sintéticos...")
        df = generate_synthetic_data()

    # --- Análise Exploratória e Visualizações ---
    display(Markdown("## Dados Carregados/Gerados"))
    print(df.info())
    print(df.head())
    print(df.describe(include='all'))

    # --- Gráficos (11 ao total, 4 tipos) ---

    # 1. Gráfico de Linha (Desempenho ao longo do tempo)
    plot_line(df, 'aluno_id', 'desempenho', "Desempenho ao Longo do Tempo", "desempenho_line.png", graficos_drive_path, hue='grupo')

    # 2. Boxplot (Satisfação por Grupo)
    plot_boxplot(df, 'grupo', 'satisfacao', "Satisfação por Grupo", "satisfacao_boxplot.png", graficos_drive_path)

    # 3. Boxplot (Pertencimento por Grupo)
    plot_boxplot(df, 'grupo', 'pertencimento', "Pertencimento por Grupo", "pertencimento_boxplot.png", graficos_drive_path)

    # 4. Violin Plot (Desempenho por Grupo, com split)
    plot_violin(df, 'grupo', 'desempenho', "Distribuição de Desempenho por Grupo (Violino)", "desempenho_violin.png", graficos_drive_path, hue='grupo', split=True)

    # 5. Violin Plot (Participação no Fórum por Grupo)
    plot_violin(df, 'grupo', 'participacao_forum', "Participação no Fórum por Grupo (Violino)", "participacao_violin.png", graficos_drive_path, hue='grupo')

    # 6. KDE Plot (Desempenho, com hue por grupo)
    plot_kde(df, 'desempenho', "Densidade de Desempenho", "desempenho_kde.png", graficos_drive_path, hue='grupo')

    # 7. KDE Plot (Centralidade de Grau)
    plot_kde(df, 'centralidade_grau', 'Distribuição da Centralidade', 'centralidade_kde.png', graficos_drive_path)

    # 8. KDE Plot (Participação no Fórum, com hue por grupo)
    plot_kde(df, 'participacao_forum', "Densidade de Participação no Fórum", "participacao_kde.png", graficos_drive_path, hue='grupo', multiple="stack")

    # 9. KDE Plot (Interações com Colegas, com hue por grupo)
    plot_kde(df, 'interacoes_colegas', "Densidade de Interações com Colegas", "interacoes_kde.png", graficos_drive_path, hue='grupo', multiple="fill")

    # --- Análises (Opcional, para dados adicionais)
    cluster_features = ['disciplina_1', 'disciplina_2', 'disciplina_3',
                        'participacao_forum', 'interacoes_colegas', 'desempenho']
    df = analyze_clusters(df, cluster_features)  # Análise de cluster (opcional)

    # 10. Violin Plot (Desempenho por Cluster)
    if 'cluster' in df.columns:
        plot_violin(df, 'cluster', 'desempenho', "Desempenho por Cluster (Violino)", "desempenho_cluster_violin.png", graficos_drive_path)

    # 11. Boxplot (Interações com Colegas por Grupo)
    plot_boxplot(df, 'grupo', 'interacoes_colegas', "Interações com Colegas por Grupo", "interacoes_boxplot.png", graficos_drive_path)


    graph, df = analyze_social_network(df)  # Análise de rede social (opcional)
    df = create_recommendations(df) # Recomendação (opcional)

    display(Markdown("## Conclusões (Exemplo)"))
    display(Markdown("Resultados preliminares (dados carregados ou sintéticos)."))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Análise do Modelo Metateca

## Dados Carregados/Gerados

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   aluno_id            40 non-null     int64  
 1   grupo               40 non-null     object 
 2   disciplina_1        40 non-null     int64  
 3   disciplina_2        40 non-null     int64  
 4   disciplina_3        40 non-null     int64  
 5   participacao_forum  40 non-null     float64
 6   interacoes_colegas  40 non-null     int64  
 7   desempenho          40 non-null     float64
 8   satisfacao          40 non-null     int64  
 9   pertencimento       40 non-null     int64  
 10  centralidade_grau   40 non-null     float64
 11  recomendacao        40 non-null     object 
dtypes: float64(3), int64(7), object(2)
memory usage: 3.9+ KB
None
   aluno_id        grupo  disciplina_1  disciplina_2  disciplina_3  \
0         1  Com_Agentes             1             0             1   
1   

<ipython-input-7-6dd7a3032533>:77: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(x=x_col, y=y_col, data=df, palette=DEFAULT_PALETTE, hue=hue)


...

<ipython-input-7-6dd7a3032533>:77: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(x=x_col, y=y_col, data=df, palette=DEFAULT_PALETTE, hue=hue)


Gráfico salvo: /content/drive/MyDrive/graficos/interacoes_boxplot.png

--- Análise de Rede Social ---

--- Sistema de Recomendação (Simulado) ---
Recomendações (5 primeiros):
    aluno_id  recomendacao
0         1  disciplina_2
1         2  disciplina_1
2         3  disciplina_3
3         4  disciplina_2
4         5  disciplina_2


<ipython-input-7-6dd7a3032533>:166: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['centralidade_grau'].fillna(0, inplace=True)


## Conclusões (Exemplo)

Resultados preliminares (dados carregados ou sintéticos).